In [1]:
import pandas as pd
import json
from datetime import datetime
import random
import copy

In [2]:
D = {}
with open('data.jl',encoding='utf-8') as f:
    for l in f.readlines():
        try:
            j = json.loads(l)
            pkgname = j['pkg']
            D[pkgname] = j
        except:
            pass
len(D)

642744

In [3]:
D['com.facebook.orca']

{'name': 'Messenger – Text and Video Chat for Free',
 'pkg': 'com.facebook.orca',
 'publisher': 'Facebook',
 'genres': ['Communication'],
 'content_ratings': ['Everyone'],
 'is_editor_choice': True,
 'price': 0.0,
 'inapp_details': ['Offers in-app purchases'],
 'has_video': False,
 'num_screenshot': 8,
 'description': "* SEND A MESSAGE -Skip exchanging phone numbers, just send a message. It's seamless across devices.\n* SHOW YOUR REACTION - Add a silly sticker, send a dancing GIF and use emojis to express yourself better.\n* SHARE PHOTOS AND VIDEOS - Capture the moment with the Messenger Camera, add a filter or doodle, then send to friends or share to Stories.\n* RALLY THE GROUP - Make plans to get together with polls, share your location, split the bill and more.\n* HOP ON A CALL OR VIDEO CHAT - Hang out anywhere, anytime by getting on call or a group video** chat — filters optional :) **Calls are free over Wi-Fi but otherwise standard data charges apply.\n* SEND A VOICE MESSAGE - Whe

In [4]:
# D1 expands meaningful fields in additional_info out as top level keys, gets rid of description
D1 = dict()
for pkg in D:
    j = D[pkg]
    new_j = dict()
    for tlkey in j: # top level key of the line of json denoting an object
        if tlkey == 'additional_info':
            for aikey in j[tlkey]: # key inside 'additional_info':
                # filter out useless additional info
                if aikey not in ['Report', 'Permissions', 'Content Rating', 
                                 'Developer', 'Offered By', 'Eligible for Family Library']: 
                    new_j[aikey] = j[tlkey][aikey]
        elif tlkey == 'description': # no need for description in the essential csv
            continue
        else:
            new_j[tlkey] = j[tlkey]
    D1[pkg] = new_j
len(D1)

642744

In [5]:
D1['com.facebook.orca']

{'name': 'Messenger – Text and Video Chat for Free',
 'pkg': 'com.facebook.orca',
 'publisher': 'Facebook',
 'genres': ['Communication'],
 'content_ratings': ['Everyone'],
 'is_editor_choice': True,
 'price': 0.0,
 'inapp_details': ['Offers in-app purchases'],
 'has_video': False,
 'num_screenshot': 8,
 'rating_overall': 4.1,
 'rating_5': 40250992,
 'rating_4': 9365883,
 'rating_3': 6007039,
 'rating_2': 2502559,
 'rating_1': 7830646,
 'rating_count': 65957119,
 'Updated': 'April 23, 2019',
 'Size': 'Varies with device',
 'Installs': '1,000,000,000+',
 'Current Version': 'Varies with device',
 'Requires Android': 'Varies with device',
 'Interactive Elements': 'Users Interact, Shares Location, Digital Purchases',
 'In-app Products': '$0.99 - $399.99 per item'}

In [6]:
# there can be 2 genres and 2 content_rating, multiple inapp details 
# extracts it into a single field as the first one in the list in case there are two and save it in place in D1
for pkg in D1:
    j = D1[pkg]
    if len(j['genres']) == 0:
        j['genres'] = 'unknown'
    else:
        j['genres'] = j['genres'][0]
    j['content_ratings'] = j['content_ratings'][0]
    if len(j['inapp_details']) == 0:
        j['inapp_details'] = 'null'
    else:
        j['inapp_details'] = j['inapp_details'][0]
len(D1)

642744

In [7]:
D1['com.facebook.orca']

{'name': 'Messenger – Text and Video Chat for Free',
 'pkg': 'com.facebook.orca',
 'publisher': 'Facebook',
 'genres': 'Communication',
 'content_ratings': 'Everyone',
 'is_editor_choice': True,
 'price': 0.0,
 'inapp_details': 'Offers in-app purchases',
 'has_video': False,
 'num_screenshot': 8,
 'rating_overall': 4.1,
 'rating_5': 40250992,
 'rating_4': 9365883,
 'rating_3': 6007039,
 'rating_2': 2502559,
 'rating_1': 7830646,
 'rating_count': 65957119,
 'Updated': 'April 23, 2019',
 'Size': 'Varies with device',
 'Installs': '1,000,000,000+',
 'Current Version': 'Varies with device',
 'Requires Android': 'Varies with device',
 'Interactive Elements': 'Users Interact, Shares Location, Digital Purchases',
 'In-app Products': '$0.99 - $399.99 per item'}

In [8]:
# converts time to yyyy-mm-dd in place in D1
for pkg in D1:
    j = D1[pkg]
    t = datetime.strptime(j['Updated'], '%B %d, %Y')
    j['Updated'] = t.strftime('%Y-%m-%d')

In [9]:
D1['com.facebook.orca']

{'name': 'Messenger – Text and Video Chat for Free',
 'pkg': 'com.facebook.orca',
 'publisher': 'Facebook',
 'genres': 'Communication',
 'content_ratings': 'Everyone',
 'is_editor_choice': True,
 'price': 0.0,
 'inapp_details': 'Offers in-app purchases',
 'has_video': False,
 'num_screenshot': 8,
 'rating_overall': 4.1,
 'rating_5': 40250992,
 'rating_4': 9365883,
 'rating_3': 6007039,
 'rating_2': 2502559,
 'rating_1': 7830646,
 'rating_count': 65957119,
 'Updated': '2019-04-23',
 'Size': 'Varies with device',
 'Installs': '1,000,000,000+',
 'Current Version': 'Varies with device',
 'Requires Android': 'Varies with device',
 'Interactive Elements': 'Users Interact, Shares Location, Digital Purchases',
 'In-app Products': '$0.99 - $399.99 per item'}

In [10]:
# convert installs by stripping the + and ,s then into an int
for pkg in D1:
    j = D1[pkg]
    n = j['Installs'].replace(',','').replace('+','')
    j['Installs'] = int(n)

In [11]:
D1['com.facebook.orca']

{'name': 'Messenger – Text and Video Chat for Free',
 'pkg': 'com.facebook.orca',
 'publisher': 'Facebook',
 'genres': 'Communication',
 'content_ratings': 'Everyone',
 'is_editor_choice': True,
 'price': 0.0,
 'inapp_details': 'Offers in-app purchases',
 'has_video': False,
 'num_screenshot': 8,
 'rating_overall': 4.1,
 'rating_5': 40250992,
 'rating_4': 9365883,
 'rating_3': 6007039,
 'rating_2': 2502559,
 'rating_1': 7830646,
 'rating_count': 65957119,
 'Updated': '2019-04-23',
 'Size': 'Varies with device',
 'Installs': 1000000000,
 'Current Version': 'Varies with device',
 'Requires Android': 'Varies with device',
 'Interactive Elements': 'Users Interact, Shares Location, Digital Purchases',
 'In-app Products': '$0.99 - $399.99 per item'}

In [12]:
# finally change the key names of things as following
# save to D2
mapping = {
    'Updated': 'update_time',
    'Size': 'size',
    'Installs': 'min_install', # because we got rid of the + sign
    'Current Version': 'version',
    'Requires Android': 'android_version',
    'Interactive Elements': 'interactive_elements',
    'In-app Products': 'in_app_products',
}

D2 = dict()
for pkg in D1:
    j = D1[pkg]
    nj = dict()
    for key in j:
        if key in mapping:
            nj[mapping[key]] = j[key]
        else:
            nj[key] = j[key]
    D2[pkg] = nj

In [13]:
D2['com.facebook.orca']

{'name': 'Messenger – Text and Video Chat for Free',
 'pkg': 'com.facebook.orca',
 'publisher': 'Facebook',
 'genres': 'Communication',
 'content_ratings': 'Everyone',
 'is_editor_choice': True,
 'price': 0.0,
 'inapp_details': 'Offers in-app purchases',
 'has_video': False,
 'num_screenshot': 8,
 'rating_overall': 4.1,
 'rating_5': 40250992,
 'rating_4': 9365883,
 'rating_3': 6007039,
 'rating_2': 2502559,
 'rating_1': 7830646,
 'rating_count': 65957119,
 'update_time': '2019-04-23',
 'size': 'Varies with device',
 'min_install': 1000000000,
 'version': 'Varies with device',
 'android_version': 'Varies with device',
 'interactive_elements': 'Users Interact, Shares Location, Digital Purchases',
 'in_app_products': '$0.99 - $399.99 per item'}

In [14]:
D2['com.mojang.minecraftpe']

{'name': 'Minecraft',
 'pkg': 'com.mojang.minecraftpe',
 'publisher': 'Mojang',
 'genres': 'Arcade',
 'content_ratings': 'Everyone 10+',
 'is_editor_choice': True,
 'price': 6.99,
 'inapp_details': 'Offers in-app purchases',
 'has_video': True,
 'num_screenshot': 23,
 'rating_overall': 4.5,
 'rating_5': 2088955,
 'rating_4': 243506,
 'rating_3': 119376,
 'rating_2': 58057,
 'rating_1': 167167,
 'rating_count': 2677061,
 'update_time': '2019-04-23',
 'size': 'Varies with device',
 'min_install': 10000000,
 'version': '1.11.0.23',
 'android_version': '4.2 and up',
 'interactive_elements': 'Users Interact',
 'in_app_products': '$0.99 - $49.99 per item'}

In [15]:
df = pd.DataFrame.from_dict(D2, orient='index')
df

,name,pkg,publisher,genres,content_ratings,is_editor_choice,price,inapp_details,has_video,num_screenshot,...,rating_2,rating_1,rating_count,update_time,size,min_install,version,android_version,interactive_elements,in_app_products
Air.Density,Air Density Calculator,Air.Density,DeNysschen,Books & Reference,Everyone,False,0.00,null,False,2,...,0,2,6,2012-04-29,111k,1000,1.3,2.0 and up,NaN,NaN
Air.Lite,Air Lite Psychrometric Calcs,Air.Lite,"Cold Means Mobile Software, LLC",Productivity,Everyone,False,0.00,Contains Ads,False,4,...,17,28,390,2014-06-17,1.2M,100000,1.1,2.3 and up,NaN,NaN
Alekseyt.Lusher,Тест Люшера,Alekseyt.Lusher,Alekseyt,Entertainment,Everyone,False,0.00,Contains Ads,False,3,...,304,794,8190,2015-01-24,1.6M,1000000,1.6,2.1 and up,NaN,NaN
Alfasoft.Widgets.PsychedelicAnalogClock,Psychedelic Clock,Alfasoft.Widgets.PsychedelicAnalogClock,Alfasoft,Personalization,Everyone,False,0.00,null,False,2,...,28,59,416,2014-01-09,96k,50000,1.3,1.5 and up,NaN,NaN
Alfasoft.Widgets.SweetYorkshireAnalogClock,Sweet Yorkshire Clock,Alfasoft.Widgets.SweetYorkshireAnalogClock,Alfasoft,Personalization,Everyone,False,0.00,null,False,2,...,11,53,285,2014-01-18,79k,10000,1.1,1.5 and up,NaN,NaN
AlligatorCallPackage.Pkg,Alligator Call,AlligatorCallPackage.Pkg,FolsomApps,Sports,Everyone,False,0.99,null,False,2,...,0,1,7,2011-01-26,2.4M,500,1.0,1.1 and up,NaN,NaN
AmaxSoftware.bubblelevel,Bubble Level (Scale),AmaxSoftware.bubblelevel,LeXus22,Tools,Everyone,False,0.00,null,False,3,...,1,3,19,2012-01-23,634k,5000,1.0.3,2.1 and up,NaN,NaN
Andrew.Compass.com,Rapid Compass,Andrew.Compass.com,Andrew Android Apps,Tools,Everyone,False,0.00,null,True,8,...,97,368,1720,2013-11-15,231k,500000,1.4,2.2 and up,NaN,NaN
Android.WirelessPassword,Wifi Password Recovery,Android.WirelessPassword,Osama Abukmail,Tools,Everyone,False,0.00,Contains Ads,False,4,...,138,954,3760,2017-12-07,2.2M,500000,2.6,4.0.3 and up,NaN,NaN
Animato.Eained,E-ained,Animato.Eained,Animato,Books & Reference,Everyone,False,0.00,Contains Ads,False,3,...,26,29,550,2012-09-02,179k,50000,1.1,1.6 and up,NaN,NaN


In [16]:
df_sorted = df.sort_values(by=['rating_count'], ascending=False)
df_sorted

,name,pkg,publisher,genres,content_ratings,is_editor_choice,price,inapp_details,has_video,num_screenshot,...,rating_2,rating_1,rating_count,update_time,size,min_install,version,android_version,interactive_elements,in_app_products
com.whatsapp,WhatsApp Messenger,com.whatsapp,WhatsApp Inc.,Communication,Everyone,False,0.00,null,False,6,...,2118908,5344252,87499059,2019-04-24,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Digital Purchases",NaN
com.facebook.katana,Facebook,com.facebook.katana,Facebook,Social,Teen,False,0.00,Contains Ads·Offers in-app purchases,False,5,...,3499060,10113178,86433210,2019-04-25,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Shares Info, Shares Location, ...",$0.99 - $399.99 per item
com.instagram.android,Instagram,com.instagram.android,Instagram,Social,Teen,True,0.00,Contains Ads,False,5,...,1448081,3356089,80808062,2019-04-23,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Shares Info, Shares Location",NaN
com.facebook.orca,Messenger – Text and Video Chat for Free,com.facebook.orca,Facebook,Communication,Everyone,True,0.00,Offers in-app purchases,False,8,...,2502559,7830646,65957119,2019-04-23,Varies with device,1000000000,Varies with device,Varies with device,"Users Interact, Shares Location, Digital Purch...",$0.99 - $399.99 per item
com.supercell.clashofclans,Clash of Clans,com.supercell.clashofclans,Supercell,Strategy,Everyone 10+,True,0.00,Offers in-app purchases,True,21,...,737549,2093231,48650861,2019-04-23,103M,500000000,11.446.24,4.1 and up,"Users Interact, Digital Purchases",$0.99 - $100.00 per item
com.cleanmaster.mguard,"Clean Master - Antivirus, Applock & Cleaner",com.cleanmaster.mguard,Cheetah Mobile,Tools,Everyone,False,0.00,Contains Ads·Offers in-app purchases,True,6,...,519694,975487,44229745,2019-04-22,20M,1000000000,7.0.8,Varies with device,NaN,$2.99 - $10.68 per item
com.google.android.youtube,YouTube,com.google.android.youtube,Google LLC,Video Players & Editors,Teen,True,0.00,Contains Ads,False,5,...,1092518,3363947,43487619,2019-04-19,Varies with device,5000000000,Varies with device,Varies with device,"Users Interact, Digital Purchases",NaN
com.kiloo.subwaysurf,Subway Surfers,com.kiloo.subwaysurf,Kiloo,Arcade,Everyone 10+,True,0.00,Contains Ads·Offers in-app purchases,True,25,...,695792,1516050,29993489,2019-04-25,88M,1000000000,1.102.0,4.1 and up,Digital Purchases,$0.99 - $99.99 per item
com.supercell.clashroyale,Clash Royale,com.supercell.clashroyale,Supercell,Strategy,Everyone 10+,True,0.00,Offers in-app purchases,True,19,...,331601,1490455,25698937,2019-04-15,87M,100000000,2.7.1,4.1 and up,"Users Interact, Digital Purchases",$0.99 - $99.99 per item
com.cleanmaster.security,"Security Master - Antivirus, VPN, AppLock, Boo...",com.cleanmaster.security,Cheetah Mobile (AppLock & AntiVirus),Tools,Everyone,False,0.00,Contains Ads·Offers in-app purchases,False,7,...,296582,531227,25579290,2019-04-22,22M,500000000,4.9.3,Varies with device,Digital Purchases,$0.99 - $35.99 per item


In [17]:
df_sorted.to_csv('data_essential.csv', index=False)